In [1]:
import os
import math
import json

In [2]:
def aoc_2021_18_1(file_path):
    """--- Day 18: Snailfish --- Part One"""
    
    class SnailNode():
        """Node of snailfish values"""

        def __init__(self):
            self.parent = None
            self.left_sibling = None
            self.right_sibling = None
            self.left_child = None
            self.right_child = None
            self.value = None
        
    def visualize_snail(snail_tree):
        """Visualize snail tree as a nested list"""

        if snail_tree.value is not None:
            return snail_tree.value
        return [visualize_snail(snail_tree.left_child), visualize_snail(snail_tree.right_child)]

    def init_snail(snail_list):
        """Initialize a list of snails, return ancestor node of binary tree"""

        snail_tree = SnailNode()

        # whether left child is leaf
        if type(snail_list[0]) is int:
            left_child = SnailNode()
            left_child.value = snail_list[0]
        else:
            left_child = init_snail(snail_list[0])

        # whether right child is leaf
        if type(snail_list[1]) is int:
            right_child = SnailNode()
            right_child.value = snail_list[1]
        else:
            right_child = init_snail(snail_list[1])

        # define children
        snail_tree.left_child = left_child
        snail_tree.right_child = right_child

        # define parents
        left_child.parent = snail_tree
        right_child.parent = snail_tree

        # define siblings
        left_child.right_sibling = right_child
        right_child.left_sibling = left_child

        return snail_tree

    def add_snail(snail1, snail2):
        """Addition of two snailfish numbers"""

        # define children
        snail_tree = SnailNode()
        snail_tree.left_child = snail1
        snail_tree.right_child = snail2

        # define siblings
        snail1.right_sibling = snail2
        snail2.left_sibling = snail1

        # define parent
        snail1.parent = snail_tree
        snail2.parent = snail_tree

        # reduce tree one step at a time if necessary
        snail_tree, action_flag = _reduce_snail(snail_tree)
        while action_flag:
            snail_tree, action_flag = _reduce_snail(snail_tree)

        return snail_tree

    def _reduce_snail(reduce_tree):
        """Reduce snail one step at a time if necessary"""

        reduce_tree, action_flag = _explode(reduce_tree)
        if action_flag is not True:
            reduce_tree, action_flag = _split(reduce_tree)
        return reduce_tree, action_flag

    def _explode(explode_tree, nest=0):
        """Function to explode nested lists"""

        # explode
        if nest >= 4 and explode_tree.value is None and explode_tree.left_child.value is not None:

            # define left value
            temp_pointer = explode_tree
            left_pointer = None
            while temp_pointer.parent is not None:
                if temp_pointer.left_sibling is not None:
                    left_pointer = temp_pointer.left_sibling      
                    while left_pointer.right_child is not None:
                        left_pointer = left_pointer.right_child
                    break
                else:
                    temp_pointer = temp_pointer.parent

            # add value to its right
            if left_pointer is not None:
                left_pointer.value += explode_tree.left_child.value

            # define right value
            temp_pointer = explode_tree
            right_pointer = None
            while temp_pointer.parent is not None:
                if temp_pointer.right_sibling is not None:
                    right_pointer = temp_pointer.right_sibling      
                    while right_pointer.left_child is not None:
                        right_pointer = right_pointer.left_child
                    break
                else:
                    temp_pointer = temp_pointer.parent

            # add value to its right
            if right_pointer is not None:
                right_pointer.value += explode_tree.right_child.value

            # explode
            explode_tree.value = 0
            explode_tree.left_child = None
            explode_tree.right_child = None

            return explode_tree, True

        # nothing to explode
        elif explode_tree.value is not None:
            return explode_tree, False

        # dfs tree
        explode_tree.left_child, action_flag = _explode(explode_tree.left_child, nest=nest+1)
        if not action_flag:
            explode_tree.right_child, action_flag = _explode(explode_tree.right_child, nest=nest+1)      

        return explode_tree, action_flag

    def _split(split_tree):
        """Function to split large values"""

        # split
        if split_tree.value is not None:
            if split_tree.value >= 10:

                # define split values
                split_tree.left_child = SnailNode()
                split_tree.left_child.value = math.floor(split_tree.value/2)
                split_tree.right_child = SnailNode()
                split_tree.right_child.value = math.ceil(split_tree.value/2)

                # define parent
                split_tree.left_child.parent = split_tree
                split_tree.right_child.parent = split_tree

                # define siblings
                split_tree.left_child.right_sibling = split_tree.right_child
                split_tree.right_child.left_sibling = split_tree.left_child

                # split value
                split_tree.value = None

                return split_tree, True
            else:
                return split_tree, False

        # dfs tree
        split_tree.left_child, action_flag = _split(split_tree.left_child)
        if not action_flag:
            split_tree.right_child, action_flag = _split(split_tree.right_child)      

        return split_tree, action_flag

    def magnitude_snail(snail_tree):
        """Recursive function to calculate snail magnitude"""

        if snail_tree.value is not None:
            return snail_tree.value
        return 3 * magnitude_snail(snail_tree.left_child) + 2 * magnitude_snail(snail_tree.right_child)
    
    with open(file_path) as f:
        aoc_read = f.read().split('\n')
    snailfish_numbers = [json.loads(f) for f in aoc_read]
    
    # add list of snail numbers
    snailfish_tree = init_snail(snailfish_numbers[0])
    for snailfish in snailfish_numbers[1:]:
        snailfish_tree = add_snail(snailfish_tree, init_snail(snailfish))
         
    snail_magnitude = magnitude_snail(snailfish_tree)
    return snail_magnitude

In [3]:
aoc_2021_18_1('example.txt')

4140

In [4]:
aoc_2021_18_1('input.txt')

4017

In [5]:
def aoc_2021_18_2(file_path):
    """--- Day 18: Snailfish --- Part Two"""
    
    class SnailNode():
        """Node of snailfish values"""

        def __init__(self):
            self.parent = None
            self.left_sibling = None
            self.right_sibling = None
            self.left_child = None
            self.right_child = None
            self.value = None
        
    def visualize_snail(snail_tree):
        """Visualize snail tree as a nested list"""

        if snail_tree.value is not None:
            return snail_tree.value
        return [visualize_snail(snail_tree.left_child), visualize_snail(snail_tree.right_child)]

    def init_snail(snail_list):
        """Initialize a list of snails, return ancestor node of binary tree"""

        snail_tree = SnailNode()

        # whether left child is leaf
        if type(snail_list[0]) is int:
            left_child = SnailNode()
            left_child.value = snail_list[0]
        else:
            left_child = init_snail(snail_list[0])

        # whether right child is leaf
        if type(snail_list[1]) is int:
            right_child = SnailNode()
            right_child.value = snail_list[1]
        else:
            right_child = init_snail(snail_list[1])

        # define children
        snail_tree.left_child = left_child
        snail_tree.right_child = right_child

        # define parents
        left_child.parent = snail_tree
        right_child.parent = snail_tree

        # define siblings
        left_child.right_sibling = right_child
        right_child.left_sibling = left_child

        return snail_tree

    def add_snail(snail1, snail2):
        """Addition of two snailfish numbers"""

        # define children
        snail_tree = SnailNode()
        snail_tree.left_child = snail1
        snail_tree.right_child = snail2

        # define siblings
        snail1.right_sibling = snail2
        snail2.left_sibling = snail1

        # define parent
        snail1.parent = snail_tree
        snail2.parent = snail_tree

        # reduce tree one step at a time if necessary
        snail_tree, action_flag = _reduce_snail(snail_tree)
        while action_flag:
            snail_tree, action_flag = _reduce_snail(snail_tree)

        return snail_tree

    def _reduce_snail(reduce_tree):
        """Reduce snail one step at a time if necessary"""

        reduce_tree, action_flag = _explode(reduce_tree)
        if action_flag is not True:
            reduce_tree, action_flag = _split(reduce_tree)
        return reduce_tree, action_flag

    def _explode(explode_tree, nest=0):
        """Function to explode nested lists"""

        # explode
        if nest >= 4 and explode_tree.value is None and explode_tree.left_child.value is not None:

            # define left value
            temp_pointer = explode_tree
            left_pointer = None
            while temp_pointer.parent is not None:
                if temp_pointer.left_sibling is not None:
                    left_pointer = temp_pointer.left_sibling      
                    while left_pointer.right_child is not None:
                        left_pointer = left_pointer.right_child
                    break
                else:
                    temp_pointer = temp_pointer.parent

            # add value to its right
            if left_pointer is not None:
                left_pointer.value += explode_tree.left_child.value

            # define right value
            temp_pointer = explode_tree
            right_pointer = None
            while temp_pointer.parent is not None:
                if temp_pointer.right_sibling is not None:
                    right_pointer = temp_pointer.right_sibling      
                    while right_pointer.left_child is not None:
                        right_pointer = right_pointer.left_child
                    break
                else:
                    temp_pointer = temp_pointer.parent

            # add value to its right
            if right_pointer is not None:
                right_pointer.value += explode_tree.right_child.value

            # explode
            explode_tree.value = 0
            explode_tree.left_child = None
            explode_tree.right_child = None

            return explode_tree, True

        # nothing to explode
        elif explode_tree.value is not None:
            return explode_tree, False

        # dfs tree
        explode_tree.left_child, action_flag = _explode(explode_tree.left_child, nest=nest+1)
        if not action_flag:
            explode_tree.right_child, action_flag = _explode(explode_tree.right_child, nest=nest+1)      

        return explode_tree, action_flag

    def _split(split_tree):
        """Function to split large values"""

        # split
        if split_tree.value is not None:
            if split_tree.value >= 10:

                # define split values
                split_tree.left_child = SnailNode()
                split_tree.left_child.value = math.floor(split_tree.value/2)
                split_tree.right_child = SnailNode()
                split_tree.right_child.value = math.ceil(split_tree.value/2)

                # define parent
                split_tree.left_child.parent = split_tree
                split_tree.right_child.parent = split_tree

                # define siblings
                split_tree.left_child.right_sibling = split_tree.right_child
                split_tree.right_child.left_sibling = split_tree.left_child

                # split value
                split_tree.value = None

                return split_tree, True
            else:
                return split_tree, False

        # dfs tree
        split_tree.left_child, action_flag = _split(split_tree.left_child)
        if not action_flag:
            split_tree.right_child, action_flag = _split(split_tree.right_child)      

        return split_tree, action_flag

    def magnitude_snail(snail_tree):
        """Recursive function to calculate snail magnitude"""

        if snail_tree.value is not None:
            return snail_tree.value
        return 3 * magnitude_snail(snail_tree.left_child) + 2 * magnitude_snail(snail_tree.right_child)
    
    with open(file_path) as f:
        aoc_read = f.read().split('\n')
    snailfish_numbers = [json.loads(f) for f in aoc_read]
    
    # add list of snail numbers
    snailfish_tree = init_snail(snailfish_numbers[0])
    for snailfish in snailfish_numbers[1:]:
        snailfish_tree = add_snail(snailfish_tree, init_snail(snailfish))
         
    # find largest magnitude
    max_magnitude = 0
    for i_snail1 in range(len(snailfish_numbers) - 1):
        for i_snail2 in range(i_snail1 + 1, len(snailfish_numbers)):
            temp_sum1 = magnitude_snail(add_snail(init_snail(snailfish_numbers[i_snail1]), init_snail(snailfish_numbers[i_snail2])))
            temp_sum2 = magnitude_snail(add_snail(init_snail(snailfish_numbers[i_snail2]), init_snail(snailfish_numbers[i_snail1])))
            max_magnitude = max(max_magnitude, temp_sum1, temp_sum2)

    return max_magnitude

In [6]:
aoc_2021_18_2('example.txt')

3993

In [7]:
aoc_2021_18_2('input.txt')

4583